## <u>Cuckoo Filter</u>
### Maggie Drew & Adam Gibbs

Implementation of Cuckoo Filter for estimating set membership on data streams. Implementation psuedocode from the following paper: https://www.cs.cmu.edu/~dga/papers/cuckoo-conext2014.pdf. This document also helped with figuring out Partial-Kay Cuckoo Hashing: https://williams-cs.github.io/cs358-s20/lectures/lecture10/bloom.pdf. This notebook contains the following:

1. Imports used in the notebook
2. All global variables and python data structures (lists, dictionaries, etc.) used
3. Code for Cuckoo Filter functions
4. Cuckoo filter test on simulated data stream
5. Results
6. Analysis of Results 

In [2]:
# IMPORTS
import matplotlib as plt
import numpy as np 
import pandas as pd
import gzip
import random
import math
import time

# uncomment if library is needed
# !{sys.executable} -m pip install mmh3
import mmh3


In [3]:
# VARIABLES and DATA STRUCTURES

# dataset to be used as data stream
dataset = './input.txt'
dataset2 = './vocabset2.txt'

# number of buckets in hash table
bucket_size = 10000
# number of fingerprints in each bucket
entries_per_bucket = 4

# fingerprint size (in bits)
fingerprint_size = 7
fingerprint_mod = 2**(fingerprint_size)



In [4]:
# Class containing basic Cuckoo Filter with insert, lookup, and delete functionality
# Creates a Cuckoo Filter with buckets_size buckets
# Each bucket has depth 1 and is initialized to NaN to start
# Uses mmh3 for hash function, so elements passed into functions should be strings
#    however, the element will be cast to a string, so as long as element can be a string you're good

class CuckooFilter():

    # CONSTRUCTOR
    def __init__(self, bucket_size, k=5):
        # store value specifying length of array
        self.bucket_size = bucket_size
        # make a filter of specified size
        self.buckets = np.empty(bucket_size)
        # initialize all elements to be null
        # we use np.NaN for easy use
        self.buckets.fill(np.NaN)
        '''
        self.coefficients = []
        for i in range(0,k):
            self.coefficients.append(random.uniform(0,7919))
        '''
    
    # HELPER METHODS

    # creates fingerprint for element in datastream]
    # creates a fingerprint of fingerprint_size bits
    # bits get stored as integer by taking a hash mod 2^fingerprint_size 
    def create_fingerprint(self, x):
        return mmh3.hash(str(x),5) % fingerprint_mod

    # defines our first hash function
    # use mmh3 hash with seed set to 1
    def hash1(self, x):
        return mmh3.hash(str(x),1) % self.bucket_size

    # defines our second hash function
    # this is used to hash fingerprints for alternate bucket
    # use mmh3 hash with seed set to 2
    def hash2(self, x):
        return mmh3.hash(str(x),2) % self.bucket_size

    '''
    def k_ind_hash(self, x):
        hash_val = self.coefficients[0]
        for i in range(1, len(self.coefficients)):
            hash_val += self.coefficients[i]*math.pow(int(x),i)
        return round(hash_val) % self.bucket_size
    '''
        

    # method used to relocate entries when an incoming element has no free bucket
    # returns true if all old buckets are successfully relocated
    # returns false if all buckets are filled
    def relocate(self, f, bucket):

        count = 0
        while count < len(self.buckets):

            bucket = (bucket ^ self.hash2(int(f))) % self.bucket_size

            if pd.isnull(self.buckets[bucket]): 
                self.buckets[bucket] = f
                return True
            else:
                old = self.buckets[bucket]
                self.buckets[bucket] = f
                f = old
            count += 1
        
        return False
    
    # CORE METHODS - insert, lookup, and delete

    # method used to insert an element
    # returns true if the element is inserted
    #    or returns true if element has already been inserted
    # returns false if the element is not inserted
    def insert(self, x):

        # get fingerprint for element
        f = self.create_fingerprint(x)

        # find two hash function values
        # second hash function is... 
        # [hash_func(x) xor fingerprint(x)] % bucket_size
        h1 = self.hash1(x)
        h2 = (h1 ^ self.hash2(f)) % self.bucket_size

        # if first bucket is empty or x is already in it,,, add f and return true
        # elif second bucket is empty or x is already in it,,, add f and return true
        # else randomly pick bucket 1 or 2, then,
        # add f to that bucket and relocate the value it replaces, 
        #    if relocation is successful,,, return true
        #    if relocation is not successful,,, undo changes to buckets and return false
        if pd.isnull(self.buckets[h1]) or self.buckets[h1] == f:
            self.buckets[h1] = f
            return True
        elif pd.isnull(self.buckets[h2]) or self.buckets[h2] == f:
            self.buckets[h2] = f
            return True
        else:
            rand_num = random.random()
            if rand_num < 0.5:
                # store buckets in case relocation is unsuccessful
                old_buckets = np.copy(self.buckets)
                # swap fingerprint with current entry
                old = self.buckets[h1]
                self.buckets[h1] = f
                # relocate current entry
                added = self.relocate(old, h1)
                
                # if relocation is unsuccessful, restore buckets return false
                if not added:
                    self.buckets = old_buckets
                    return added
                else:
                    return added
            else:
                # store buckets in case relocation is unsuccessful
                old_buckets = np.copy(self.buckets)
                # swap fingerprint with current entry
                old = self.buckets[h2]
                self.buckets[h2] = f
                # relocate current entry
                added = self.relocate(old, h2)
                
                # if relocation is unsuccessful, restore buckets return false
                if not added:
                    self.buckets = old_buckets
                    return added
                else:
                    return added

    # method used to determine if an element is in the filter
    # gets the fingerprint and calcuates the hash functions, then 
    # if element is in first or second bucket,,, return true
    # otherwise,,, return false
    def lookup(self, x):
        # get fingerprint
        f = self.create_fingerprint(x)

        # get two possible buckets for element
        h1 = self.hash1(x)
        h2 = (h1 ^ self.hash2(f)) % self.bucket_size

        # if element's fingerprint is stored in either bucket,,, return true
        # otherwise,,, return false
        if self.buckets[h1] == f or self.buckets[h2] == f:
            return True
        else:
            return False

    # method used to delete an element in the filter
    # gets the fingerprint and calcuates the hash functions, then 
    # if element is in first bucket, put a NaN in that bucket,,, return true 
    # if element is in second bucket, put a NaN in that bucket,,, return true 
    # otherwise element was not in filter,,, return false
    def delete(self, x):
        # get fingerprint of the element
        f = self.create_fingerprint(x)

        # get two possible buckets of the element
        h1 = self.hash1(x)
        h2 = (h1 ^ self.hash2(f)) % self.bucket_size

        # if element is in first bucket, 
        # make that value null (np.NaN),,, return true
        # else if element is in second bucket,
        # make that value null (np.NaN),,, return true
        if self.buckets[h1] == f:
            self.buckets[h1] = np.NaN
            return True
        elif self.buckets[h2] == f:
            self.buckets[h2] = np.NaN
            return True
        
        # if element was not in either bucket, it is not in the filter,
        # thus it cannot be deleted,,, return false
        return False


In [5]:
# Class containing Extended Cuckoo Filter with insert, lookup, and delete functionality
# Creates a Cuckoo Filter with buckets_size buckets that each contain entries_per_bucket fingerprints
# Each bucket has depth 1 and is initialized to NaN to start
# Uses mmh3 for hash function, so elements passed into functions should be strings
#    however, the element will be cast to a string, so as long as element can be a string you're good

class ExtendedCuckooFilter():


    # CONSTRUCTOR
    def __init__(self, bucket_size, entries_per_bucket, k=5):
        # store values specifying shape of array
        self.bucket_size = bucket_size
        self.entries_per_bucket = entries_per_bucket
        # make a 2D array of length bucket_size 
        # of arrays of length entries_per_bucket
        self.buckets = np.empty((bucket_size, entries_per_bucket))
        # initialize all elements to be null
        # we use np.NaN for easy use
        self.buckets.fill(np.NaN)
        '''
        self.coefficients = []
        for i in range(0,k):
            self.coefficients.append(random.uniform(0,7919))
        '''
    
    # HELPER METHODS

    # creates fingerprint for element in datastream]
    # creates a fingerprint of fingerprint_size bits
    # bits get stored as integer by taking a hash mod 2^fingerprint_size 
    def create_fingerprint(self, x):
        return mmh3.hash(str(x),5) % fingerprint_mod

    # defines our first hash function
    # use mmh3 hash with seed set to 1
    def hash1(self, x):
        return mmh3.hash(str(x),1) % self.bucket_size

    # defines our second hash function
    # this is used to hash fingerprints for alternate bucket
    # use mmh3 hash with seed set to 2
    def hash2(self, x):
        return mmh3.hash(str(x),2) % self.bucket_size

    '''
    def k_ind_hash(self, x):
        hash_val = self.coefficients[0]
        for i in range(1, len(self.coefficients)):
            hash_val += self.coefficients[i]*math.pow(int(x),i)
        return round(hash_val) % self.bucket_size
    '''

    # method used to relocate entries when an incoming element has no free bucket
    # returns true if all old buckets are successfully relocated
    # returns false if all buckets are filled
    def relocate(self, f, bucket):

        count = 0
        while count < len(self.buckets):
            
            # get alternate bucket 
            bucket = (bucket ^ self.hash2(int(f))) % self.bucket_size

            # check all entries in each bucket,
            # if an entry is empty, add f
            for i in range(0, self.entries_per_bucket):
                if pd.isnull(self.buckets[bucket][i]): 
                    self.buckets[bucket][i] = f
                    return True
            
            # if entire bucket is filled,
            # randomly swap f with an entry and,
            # repeat relocation process with removed entry
            randIndex = random.randint(0,self.entries_per_bucket-1)
            old = self.buckets[bucket][randIndex]
            self.buckets[bucket][randIndex] = f
            f = old

            # increment count
            count += 1
        
        # if we looped through all buckets and entry cannot be added,
        # return false because bucket is too full
        return False
    

    # CORE METHODS - insert, lookup, and delete

    # method used to insert an element
    # returns true if the element is inserted
    #    or returns true if element has already been inserted
    # returns false if the element is not inserted
    def insert(self, x):

        # get fingerprint for element
        f = self.create_fingerprint(x)

        # find two hash function values
        # second hash function is... 
        # [hash1(x) xor hash2(fingerprint(x))] % bucket_size
        h1 = self.hash1(x)
        h2 = (h1 ^ self.hash2(f)) % self.bucket_size

        # if first bucket is empty or x is already in it,,, add f and return true
        # elif second bucket is empty or x is already in it,,, add f and return true
        # else randomly pick bucket 1 or 2, then,
        # add f to that bucket and relocate the value it replaces, 
        #    if relocation is successful,,, return true
        #    if relocation is not successful,,, undo changes to buckets and return false
        for i in range(0, self.entries_per_bucket):
            if pd.isnull(self.buckets[h1][i]) or self.buckets[h1][i] == f:
                self.buckets[h1][i] = f
                return True
        for i in range(0, self.entries_per_bucket):
            if pd.isnull(self.buckets[h2][i]) or self.buckets[h2][i] == f:
                self.buckets[h2][i] = f
                return True
        
        rand_num = random.random()
        if rand_num < 0.5:
            # store buckets in case relocation is unsuccessful
            old_buckets = np.copy(self.buckets)
            # pick random entry
            randIndex = random.randint(0,self.entries_per_bucket-1)
            # swap fingerprint with random entry
            old = self.buckets[h1][randIndex]
            self.buckets[h1][randIndex] = f
            # relocate randomly removed entry
            added = self.relocate(old, h1)
            
            # if relocation is unsuccessful, restore buckets return false
            if not added:
                self.buckets = old_buckets
                return added
            else:
                return added
        else:
            # store buckets in case relocation is unsuccessful
            old_buckets = np.copy(self.buckets)
            # pick random entry
            randIndex = random.randint(0,self.entries_per_bucket-1)
            # swap fingerprint with random entry
            old = self.buckets[h2][randIndex]
            self.buckets[h2][randIndex] = f
            # relocate randomly removed entry
            added = self.relocate(old, h2)
            
            # if relocation is unsuccessful, restore buckets return false
            if not added:
                self.buckets = old_buckets
                return added
            else:
                return added

    # method used to determine if an element is in the filter
    # gets the fingerprint and calcuates the hash functions, then 
    # if element is in first or second bucket,,, return true
    # otherwise,,, return false
    def lookup(self, x):
        # get fingerprint of element
        f = self.create_fingerprint(x)

        # get two possible buckets
        h1 = self.hash1(x)
        h2 = (h1 ^ self.hash2(f)) % self.bucket_size

        # if the element's fingerprint is an entry in either bucket,,, return true
        for i in range(0, self.entries_per_bucket):
            if self.buckets[h1][i] == f or self.buckets[h2][i] == f:
                return True
        return False

    # method used to delete an element in the filter
    # gets the fingerprint and calcuates the hash functions, then 
    # if element is in first bucket, put a NaN in that bucket,,, return true 
    # if element is in second bucket, put a NaN in that bucket,,, return true 
    # otherwise element was not in filter,,, return false
    def delete(self, x):
        # get fingerprint of element
        f = self.create_fingerprint(x)

        # get two possible buckets
        h1 = self.hash1(x)
        h2 = (h1 ^ self.hash2(f)) % self.bucket_size

        # element can only be in one of the buckets, 
        # check first bucket first
        # loop through first bucket and if element's fingerprint is an entry
        # make that entry value null (np.NaN in this implementation)
        for i in range(0, self.entries_per_bucket):
            if self.buckets[h1][i] == f:
                self.buckets[h1][i] = np.NaN
                return True

        # now loop through second bucket and if element's fingerprint is an entry
        # make that entry value null (np.NaN in this implementation)
        for i in range(0, self.entries_per_bucket):
            if self.buckets[h2][i] == f:
                self.buckets[h2][i] = np.NaN
                return True
        
        # if element wasn't found as an entry in either bucket,
        # then it cannot be deleted,,, return false
        return False


In [7]:
# READ IN DATA AND APPLY FILTER

cf = ExtendedCuckooFilter(30, 2)

with open(dataset, 'rt') as data_stream:
    for line in data_stream:
        for element in line.split():
            print("Insert " + str(element) + "? " + str(cf.insert(element)) + " f=" + " " + str(cf.create_fingerprint(element)))
print("\nFilter: \n" + str(cf.buckets))

Insert 5? True f= 103
Insert 4? True f= 91
Insert 9? True f= 17
Insert 11? True f= 12
Insert 33? True f= 23
Insert 46? True f= 30
Insert 77? True f= 31
Insert 4? True f= 91
Insert 9? True f= 17
Insert 13? True f= 76
Insert 42? True f= 22
Insert 155? True f= 104
Insert 207? True f= 37

Filter: 
[[ 12.  nan]
 [ nan  nan]
 [ 22.  nan]
 [ 37.  nan]
 [103.  nan]
 [ nan  nan]
 [ nan  nan]
 [ 17.  nan]
 [ nan  nan]
 [ 91.  76.]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ 30.  nan]
 [ nan  nan]
 [ nan  nan]
 [104.  nan]
 [ nan  nan]
 [ nan  nan]
 [ 31.  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ 23.  nan]
 [ nan  nan]
 [ nan  nan]]


In [6]:
# lookup every element that is added to the filter
# return true is expected as Cuckoo Filters have no false negatives
with open(dataset, 'rt') as data_stream:
    for line in data_stream:
        for element in line.split():
            if not cf.lookup(element):
                print("Lookup " + str(element) + "? " + str(cf.lookup(element)))

print()

# lookup 5 values that were not inserted into the filter
# return false expected, but there could be false positives
# however, false positives will such a low number of insertions are rare
for test in ['36', '75', '3', '0', '101']:
    print("Lookup " + str(test) + "? " + str(cf.lookup(test)))


Lookup 36? False
Lookup 75? False
Lookup 3? False
Lookup 0? False
Lookup 101? False


In [7]:
# delete 3 elements that are in the filter
# return true expected
print(cf.delete('5'))
print(cf.delete('33'))
print(cf.delete('77'))
# delete an element that is not in the filter
# return false expected
print(cf.delete('55'))

print()

# see if any deleted elements remain in the filter
# return false expected every time
print(cf.lookup('5'))
print(cf.lookup('33'))
print(cf.lookup('77'))
print(cf.lookup('55'))

True
True
True
False

False
False
False
False


In [8]:
# READ IN DATA AND APPLY FILTER

cf2 = ExtendedCuckooFilter(bucket_size, entries_per_bucket)

with open(dataset2, 'rt') as data_stream:
    for line in data_stream:
        for element in line.split():
            cf2.insert(element)
            #print("Insert " + str(element) + "? " + str(cf2.insert(element)))
#print("\nFilter: \n" + str(cf2.buckets))

In [9]:
# lookup every element that is added to the filter
# return true is expected as Cuckoo Filters have no false negatives
with open(dataset2, 'rt') as data_stream:
    for line in data_stream:
        for element in line.split():
            print("Lookup " + str(element) + "? " + str(cf2.lookup(element)))

print()

# lookup 5 values that were not inserted into the filter
# return false expected, but there could be false positives
# however, false positives will such a low number of insertions are rare
for test in ['mirror', 'hello', 'bird', 'super', 'raspberry']:
    print("Lookup " + str(test) + "? " + str(cf2.lookup(test)))

lation? True
Lookup translational? True
Lookup translationally? True
Lookup transmembrane? True
Lookup transmission? True
Lookup transmit? True
Lookup transmitted? True
Lookup transmitter? True
Lookup transmitting? True
Lookup transparency? True
Lookup transparent? True
Lookup transport? True
Lookup transpose? True
Lookup transposed? True
Lookup trap? True
Lookup trapped? True
Lookup trapping? True
Lookup trary? True
Lookup trast? True
Lookup trate? True
Lookup travel? True
Lookup traveling? True
Lookup travelling? True
Lookup traversal? True
Lookup traverse? True
Lookup traversed? True
Lookup traverses? True
Lookup traversing? True
Lookup treasury? True
Lookup treat? True
Lookup treated? True
Lookup treating? True
Lookup treatment? True
Lookup trec? True
Lookup tree? True
Lookup trees? True
Lookup tref? True
Lookup treisman? True
Lookup tremendous? True
Lookup trend? True
Lookup trepan? True
Lookup tresp? True
Lookup treue? True
Lookup trevor? True
Lookup tri? True
Lookup triad? True


In [9]:
# delete 3 elements that are in the filter
# return true expected
print(cf2.delete('aarp'))
print(cf2.delete('abortion'))
# delete an element that is not in the filter
# return false expected
print(cf2.delete('mirror'))

print()

# see if any deleted elements remain in the filter
# return false expected every time
print(cf2.lookup('aarp'))
print(cf2.lookup('abortion'))
print(cf2.lookup('mirror'))

True
True
False

False
False
False


In [21]:

cfe = ExtendedCuckooFilter(16000000, 5)

count = 0
added = 0
not_added = 0


with open('kosarak.dat', 'rt') as data_stream:
    start_time = time.time()
    for line in data_stream:
        for element in line.split():
            count += 1
            add = cfe.insert(element)
            if add:
                added += 1
            else:
                not_added += 1

end_time = time.time() - start_time
print(end_time)


33.339659452438354


In [28]:
print("Total insertions: " + str(count))
print("Successful Insertions (including entries already added: " + str(added))
print("Unsuccessfulo Insertions: " + str(not_added))
print("Average insertion time: " + str(round(end_time * 1000 / count, 5)) + str("ms"))


Total insertions: 102660
Successful Insertions (including entries already added: 102660
Unsuccessfulo Insertions: 0
Average insertion time: 0.00486ms


In [25]:

cfe2 = ExtendedCuckooFilter(300000, 5)

count = 0
added = 0
not_added = 0


with open('vocab.nytimes.txt', 'rt') as data_stream:
    start_time = time.time()
    for line in data_stream:
        for element in line.split():
            count += 1
            add = cfe2.insert(element)
            if add:
                added += 1
            else:
                not_added += 1

end_time = time.time() - start_time
print(end_time)

0.49921417236328125


In [27]:
print("Total insertions: " + str(count))
print("Successful Insertions (including entries already added): " + str(added))
print("Unsuccessful Insertions: " + str(not_added))
print("Average insertion time: " + str(round(end_time * 1000 / count, 5)) + str("ms"))

Total insertions: 102660
Successful Insertions (including entries already added): 102660
Unsuccessful Insertions: 0
Average insertion time: 0.00486ms
